In [22]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from scipy import spatial
import networkx as nx
import csv

In [6]:
df=pd.read_csv('/kaggle/input/medium-articles/medium_articles.csv')
df.head()

,title,text,url,authors,timestamp,tags
0,Mental Note Vol. 24,Photo by Josh Riemer on Unsplash\n\nMerry Chri...,https://medium.com/invisible-illness/mental-no...,['Ryan Fan'],2020-12-26 03:38:10.479000+00:00,"['Mental Health', 'Health', 'Psychology', 'Sci..."
1,Your Brain On Coronavirus,Your Brain On Coronavirus\n\nA guide to the cu...,https://medium.com/age-of-awareness/how-the-pa...,['Simon Spichak'],2020-09-23 22:10:17.126000+00:00,"['Mental Health', 'Coronavirus', 'Science', 'P..."
2,Mind Your Nose,Mind Your Nose\n\nHow smell training can chang...,https://medium.com/neodotlife/mind-your-nose-f...,[],2020-10-10 20:17:37.132000+00:00,"['Biotechnology', 'Neuroscience', 'Brain', 'We..."
3,The 4 Purposes of Dreams,Passionate about the synergy between science a...,https://medium.com/science-for-real/the-4-purp...,['Eshan Samaranayake'],2020-12-21 16:05:19.524000+00:00,"['Health', 'Neuroscience', 'Mental Health', 'P..."
4,Surviving a Rod Through the Head,"You’ve heard of him, haven’t you? Phineas Gage...",https://medium.com/live-your-life-on-purpose/s...,['Rishav Sinha'],2020-02-26 00:01:01.576000+00:00,"['Brain', 'Health', 'Development', 'Psychology..."


In [8]:
df = df[['title','text']]
df.shape

(192368, 2)

In [9]:
df = df.drop_duplicates()

In [10]:
df.shape

(190827, 2)

In [11]:
df['text']=df['text'].str.replace('\n','<|n|>')

In [12]:
sample_blog = df['text'][10]
print(sample_blog)

Photo credit: Leo Leung<|n|><|n|>People want to know why you do what you do.<|n|><|n|>If you’re an entrepreneur, your investors, customers, and employees want to know why you founded your company. Apply for a job, and recruiters ask about your career path. If you’re a freelancer, everyone is curious about how you got started.<|n|><|n|>Why do we crave origin stories? Because they reveal character. Superman is a good guy because his planet exploded, his parents sent him to Earth, and the Kents raised him to fight evil. Steve Jobs became a trusted CEO, in part, by bouncing back from failure. My dad thrived as a home builder, but only after terminating a partnership with a prominent colleague whose practices he found unethical.<|n|><|n|>In other words:<|n|><|n|>Your origin story helps people decide whether or not to trust you.<|n|><|n|>How to Craft a Powerful Origin Story: The 5 Essential Elements<|n|><|n|>In strategic messaging and positioning engagements with CEOs, as well as business st

In [16]:
from gensim.models import Word2Vec
import nltk
nltk.download('stopwords')


[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


False

In [23]:
sentences = sent_tokenize(sample_blog)
sentences_clean = [re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
stop_words = stopwords.words('english')

sentence_tokens = [[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
w2v = Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)

sentence_embeddings = [[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
max_len = max([len(tokens) for tokens in sentence_tokens])
sentence_embeddings = [np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

In [29]:
similarity_matrix = np.zeros([len(sentence_tokens),len(sentence_tokens)])
for i,row in enumerate(sentence_embeddings):
    for j,column in enumerate(sentence_embeddings):
        similarity_matrix[i][j]==1-spatial.distance.cosine(row,column)
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank(nx_graph,max_iter=600)
top_sentence = {sentence:scores[i] for i,sentence in enumerate(sentences)}
sentNeeded = round(0.25*len(sentences))-1
top = dict(sorted(top_sentence.items(),key=lambda x:x[1],reverse=True)[:sentNeeded])
summary=""
for sent in sentences:
    if sent in top.keys():
        summary +=sent
print(summary)

Photo credit: Leo Leung<|n|><|n|>People want to know why you do what you do.<|n|><|n|>If you’re an entrepreneur, your investors, customers, and employees want to know why you founded your company.Apply for a job, and recruiters ask about your career path.If you’re a freelancer, everyone is curious about how you got started.<|n|><|n|>Why do we crave origin stories?Because they reveal character.Superman is a good guy because his planet exploded, his parents sent him to Earth, and the Kents raised him to fight evil.Steve Jobs became a trusted CEO, in part, by bouncing back from failure.My dad thrived as a home builder, but only after terminating a partnership with a prominent colleague whose practices he found unethical.<|n|><|n|>In other words:<|n|><|n|>Your origin story helps people decide whether or not to trust you.<|n|><|n|>How to Craft a Powerful Origin Story: The 5 Essential Elements<|n|><|n|>In strategic messaging and positioning engagements with CEOs, as well as business storytel

In [30]:
count = 0
def generateSummary(blog):
    global count
    count +=1
    print("Summarising Blog: ",count)
    try:
        sentences = sent_tokenize(sample_blog)
        sentences_clean = [re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
        stop_words = stopwords.words('english')

        sentence_tokens = [[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean]
        w2v = Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)

        sentence_embeddings = [[w2v.wv.get_vector(word)[0] for word in words] for words in sentence_tokens]
        max_len = max([len(tokens) for tokens in sentence_tokens])
        sentence_embeddings = [np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
        
        similarity_matrix = np.zeros([len(sentence_tokens),len(sentence_tokens)])
        for i,row in enumerate(sentence_embeddings):
            for j,column in enumerate(sentence_embeddings):
                similarity_matrix[i][j]==1-spatial.distance.cosine(row,column)
        nx_graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(nx_graph,max_iter=600)
        top_sentence = {sentence:scores[i] for i,sentence in enumerate(sentences)}
        sentNeeded = round(0.25*len(sentences))-1
        top = dict(sorted(top_sentence.items(),key=lambda x:x[1],reverse=True)[:sentNeeded])
        summary=""
        for sent in sentences:
            if sent in top.keys():
                summary +=sent
        return summary
    except:
        return float('Nan')

In [ ]:
import math
filename = 'articleset.csv'
fields = ['title','summary','content']

with open(filename,'a')as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    def callback(row):
        summary = generateSummary(row['text'])
        if(type(summary)!=str):
            return
        rows = [row['title'],summary,row['text']]
        csvwriter.writerow(rows)
    df.apply(callback,axis=1)

Summarising Blog:  1
Summarising Blog:  2
Summarising Blog:  3
Summarising Blog:  4
Summarising Blog:  5
Summarising Blog:  6
Summarising Blog:  7
Summarising Blog:  8
Summarising Blog:  9
Summarising Blog:  10
Summarising Blog:  11
Summarising Blog:  12
Summarising Blog:  13
Summarising Blog:  14
Summarising Blog:  15
Summarising Blog:  16
Summarising Blog:  17
Summarising Blog:  18
Summarising Blog:  19
Summarising Blog:  20
Summarising Blog:  21
Summarising Blog:  22
Summarising Blog:  23
Summarising Blog:  24
Summarising Blog:  25
Summarising Blog:  26
Summarising Blog:  27
Summarising Blog:  28
Summarising Blog:  29
Summarising Blog:  30
Summarising Blog:  31
Summarising Blog:  32
Summarising Blog:  33
Summarising Blog:  34
Summarising Blog:  35
Summarising Blog:  36
Summarising Blog:  37
Summarising Blog:  38
Summarising Blog:  39
Summarising Blog:  40
Summarising Blog:  41
Summarising Blog:  42
Summarising Blog:  43
Summarising Blog:  44
Summarising Blog:  45
Summarising Blog:  